In [ ]:


import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from src.linear_regression_scratch import LinearRegressionScratch
from src.bias_variance import bias_variance_decomposition

X, y = load_diabetes(return_X_y=True)

# For clear visualization, take only one feature
X = X[:, [2]]  # one feature to keep polynomial simple
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


degrees = range(1, 10)
bias_sq_list, var_list, mse_total_list = [], [], []

for d in degrees:
    # Create polynomial features manually
    X_train_poly = np.hstack([X_train**i for i in range(1, d+1)])
    X_test_poly = np.hstack([X_test**i for i in range(1, d+1)])
    
    b, v, mse = bias_variance_decomposition(
        model_class=LinearRegressionScratch,
        X_train=X_train_poly, 
        y_train=y_train, 
        X_test=X_test_poly, 
        y_test=y_test,
        n_runs=80,  # number of bootstrap samples
        model_args={'fit_intercept': True}
    )
    
    bias_sq_list.append(b)
    var_list.append(v)
    mse_total_list.append(mse)


plt.figure(figsize=(9, 6))
plt.plot(degrees, bias_sq_list, marker='o', label="Bias²")
plt.plot(degrees, var_list, marker='o', label="Variance")
plt.plot(degrees, mse_total_list, marker='o', label="Total MSE")

plt.title("Bias–Variance Trade-off Curve")
plt.xlabel("Model Complexity (Polynomial Degree)")
plt.ylabel("Error")
plt.legend()
plt.grid(True)
plt.show()


for d, b, v, m in zip(degrees, bias_sq_list, var_list, mse_total_list):
    print(f"Degree {d:2d} | Bias² = {b:8.2f} | Variance = {v:8.2f} | Total MSE = {m:8.2f}")


In [ ]:


import numpy as np
import matplotlib.pyplot as plt
from src.ridge_regression_scratch import RidgeRegressionScratch
from src.bias_variance import bias_variance_decomposition
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split


X, y = load_diabetes(return_X_y=True)
X = X[:, [2]]  # single feature for clarity
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


alphas = [0.0, 0.1, 1.0, 10.0, 100.0]
bias_sq_list, var_list, mse_total_list = [], [], []

for alpha in alphas:
    model_args = {'alpha': alpha, 'fit_intercept': True}
    b, v, mse = bias_variance_decomposition(
        model_class=RidgeRegressionScratch,
        X_train=X_train, 
        y_train=y_train, 
        X_test=X_test, 
        y_test=y_test,
        model_args=model_args,
        n_runs=80
    )
    bias_sq_list.append(b)
    var_list.append(v)
    mse_total_list.append(mse)


plt.figure(figsize=(9, 6))
plt.plot(alphas, bias_sq_list, marker='o', label="Bias²")
plt.plot(alphas, var_list, marker='o', label="Variance")
plt.plot(alphas, mse_total_list, marker='o', label="Total MSE")

plt.xscale('log')
plt.title("Effect of Regularization (Ridge Regression)")
plt.xlabel("Regularization Strength (λ)")
plt.ylabel("Error")
plt.legend()
plt.grid(True)
plt.show()


print("Ridge Regression Bias–Variance Results:\n")
for alpha, b, v, m in zip(alphas, bias_sq_list, var_list, mse_total_list):
    print(f"λ = {alpha:6.2f} | Bias² = {b:8.2f} | Variance = {v:8.2f} | Total MSE = {m:8.2f}")
